## Importando Bibliotecas

In [6]:
import pandas as pd
import requests
import schedule
import time
from plyer import notification
from datetime import datetime

## Funções

In [2]:
def extrai_api_data(api_url):
    ''' 
    Extrai dados da API pela quantidade de
    páginas. Assim que uma página sem conteúdo é 
    encontrada, a função para. 
    Enquanto a função está sendo executada, ela 
    pega os dados da página em formato json e coloca
    em uma lista.

    Parametros
    ----------
    api_url : str
            url da página

    Retorna
    -------
    lista_url : list
        lista com o json de cada página

    '''
    
    lista_url = []
    i = 1

    while True:
        url = f"{api_url}/{i}/"
        response = requests.get(url)

        if response.status_code == 200:
            data_json = response.json()
            lista_url.append(data_json)
            print(f"Página {i}: Acesso permitido!")
            i += 1
        else:
            print(f"Página {i}: Erro ao acessar a API")
            break

    return lista_url


def cria_dataframe(dicio):
    '''
    Cria um dataframe baseado em um dicionário e
    um index baseado na coluna que o usuário define.
    
    Parametros
    ----------
    dicio : dict
        dicionário com dados para o dataframe

    Retorna
    -------
    df : dataframe
        dataframe criado
    '''
    
    df = pd.DataFrame(dicio)
    
    return df

In [3]:
def executa_tarefa_api(api_url):
    ''' 
    Executa uma função como o objetivo de ser utilizada
    na schedule 
    
    Parametros
    ----------
    api_url : str 
            url da api
    Retorna
    -------
    resultado : list 
            lista de paginas extraidas da funcao extrai_api_data
    '''
    
    print("Executando a tarefa")
    resultado = extrai_api_data(api_url)
    return resultado

In [4]:
def alerta(nivel, base, etapa, erro):
    '''
    Cria uma notificação usando o notification
    baseada no nível do alerta, o nome da base
    e etapa providenciadas pelo usuário.
    Adiciona também a data e hora que a notificação
    aconteceu.
    
    Parametros:
    ----------
    nivel : int
            nivel do alerta
    base : str 
            nome da base
    etapa : str
            etapa que aconteceu
    erro : str
            erro ocorrido
    '''
    
    try:
        nome_alerta = ''

        if nivel == 1:
            nome_alerta = "Baixo"
        elif nivel == 2:
            nome_alerta = "Médio"
        else:
            nome_alerta = "Alto"
        
        current_date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        notification.notify(
            title=f"ATENÇÃO: Alerta {nome_alerta}",
            message=f"Falha no carregamento da base {base} na etapa {etapa},\
            \nErro: {erro}\
            \n{current_date_time}",
            app_name="Python",
            timeout=10
        )
        
    except Exception as error:
        print(f"Erro detectado:{error}")


In [25]:
def extracao_criacao_df(url):
    '''
    Extrai a data de uma api e baseada no tipo 
    da url, extrai diferentemente as listas e 
    cria em um dataframe 

    Parametros:
    ----------
    url : str
            url da api

    Retorna
    -------
    df : dataframe
        a data extraida em formato dataframe
    '''
    
    #extrai a url
    try:
        try:
            lista_url = executa_tarefa_api(url)

            schedule.every(1).hours.do(executa_tarefa_api, url)

            while True:
                schedule.run_pending()
                time.sleep(1)
                break

        except Exception as erro:
            alerta(1, "Pokemon" if "pokemon" in url else ("Habilidades" if "ability" in url else "Tipo"), "Extração_API", erro)

        if "pokemon" in url:
            #cria as listas com as caracteristicas
            lista_pokemon_id = [pokemon["id"] for pokemon in lista_url]
            lista_pokemon_nome = [pokemon["name"] for pokemon in lista_url]
            lista_habilidades = [[ability["ability"]["name"] for ability in pokemon["abilities"]] for pokemon in lista_url]
            lista_exp_base = [pokemon["base_experience"] for pokemon in lista_url]
            lista_tipo = [[ability["type"]["name"] for ability in pokemon["types"]] for pokemon in lista_url]
            lista_altura =[pokemon["height"] for pokemon in lista_url]
            lista_peso = [pokemon["weight"] for pokemon in lista_url]

            #cria um dicionario com as listas
            dicio = {"id":lista_pokemon_id,
            "nome": lista_pokemon_nome,
            "habilidade" : lista_habilidades,
            "exp_base": lista_exp_base,
            "tipo": lista_tipo,
            "altura": lista_altura,
            "peso": lista_peso}

            df = cria_dataframe(dicio)
                
        elif "ability" in url:
            #cria as listas com as caracteristicas
            lista_tipos_id = [tipo["id"] for tipo in lista_url]
            lista_tipos_nome = [tipo["name"] for tipo in lista_url]
            lista_geracao = [tipo["generation"]["name"] for tipo in lista_url]
            lista_pokemon = [[poke["pokemon"]["name"] for poke in pokemon["pokemon"]] for pokemon in lista_url]
            
            #cria um dicionario com as listas
            dicio = {"id":lista_tipos_id,
            "nome": lista_tipos_nome,
            "geracao": lista_geracao,
            "pokemons": lista_pokemon,
            }


            df = cria_dataframe(dicio)
            

            
        elif "type" in url:
            #cria as listas com as caracteristicas
            lista_tipos_id = [tipo["id"] for tipo in lista_url]
            lista_tipos_nome = [tipo["name"] for tipo in lista_url]
            lista_geracao = [tipo["generation"]["name"] for tipo in lista_url]
            lista_pokemon = [[poke["pokemon"]["name"] for poke in pokemon["pokemon"]] for pokemon in lista_url]

            #cria um dicionario com as listas
            dicio = {"id":lista_tipos_id,
            "nome": lista_tipos_nome,
            "geracao": lista_geracao,
            "pokemons": lista_pokemon,
            }
            


            df = cria_dataframe(dicio)

        return df
    
    except Exception as erro:
        alerta(3,"APIs", "Extração", erro)

## Extraindo Dados da API

In [30]:
habilidade_raw = extracao_criacao_df("https://pokeapi.co/api/v2/ability")

Executando a tarefa
Página 1: Acesso permitido!
Página 2: Acesso permitido!
Página 3: Acesso permitido!
Página 4: Acesso permitido!
Página 5: Acesso permitido!
Página 6: Acesso permitido!
Página 7: Acesso permitido!
Página 8: Acesso permitido!
Página 9: Acesso permitido!
Página 10: Acesso permitido!
Página 11: Acesso permitido!
Página 12: Acesso permitido!
Página 13: Acesso permitido!
Página 14: Acesso permitido!
Página 15: Acesso permitido!
Página 16: Acesso permitido!
Página 17: Acesso permitido!
Página 18: Acesso permitido!
Página 19: Acesso permitido!
Página 20: Acesso permitido!
Página 21: Acesso permitido!
Página 22: Acesso permitido!
Página 23: Acesso permitido!
Página 24: Acesso permitido!
Página 25: Acesso permitido!
Página 26: Acesso permitido!
Página 27: Acesso permitido!
Página 28: Acesso permitido!
Página 29: Acesso permitido!
Página 30: Acesso permitido!
Página 31: Acesso permitido!
Página 32: Acesso permitido!
Página 33: Acesso permitido!
Página 34: Acesso permitido!
Pág

In [27]:
tipo_raw = extracao_criacao_df("https://pokeapi.co/api/v2/type")

Executando a tarefa
Página 1: Acesso permitido!
Página 2: Acesso permitido!
Página 3: Acesso permitido!
Página 4: Acesso permitido!
Página 5: Acesso permitido!
Página 6: Acesso permitido!
Página 7: Acesso permitido!
Página 8: Acesso permitido!
Página 9: Acesso permitido!
Página 10: Acesso permitido!
Página 11: Acesso permitido!
Página 12: Acesso permitido!
Página 13: Acesso permitido!
Página 14: Acesso permitido!
Página 15: Acesso permitido!
Página 16: Acesso permitido!
Página 17: Acesso permitido!
Página 18: Acesso permitido!
Página 19: Erro ao acessar a API


In [33]:
pokemon_raw = extracao_criacao_df("https://pokeapi.co/api/v2/pokemon")

Executando a tarefa
Página 1: Acesso permitido!
Página 2: Acesso permitido!
Página 3: Acesso permitido!
Página 4: Acesso permitido!
Página 5: Acesso permitido!
Página 6: Acesso permitido!
Página 7: Acesso permitido!
Página 8: Acesso permitido!
Página 9: Acesso permitido!
Página 10: Acesso permitido!
Página 11: Acesso permitido!
Página 12: Acesso permitido!
Página 13: Acesso permitido!
Página 14: Acesso permitido!
Página 15: Acesso permitido!
Página 16: Acesso permitido!
Página 17: Acesso permitido!
Página 18: Acesso permitido!
Página 19: Acesso permitido!
Página 20: Acesso permitido!
Página 21: Acesso permitido!
Página 22: Acesso permitido!
Página 23: Acesso permitido!
Página 24: Acesso permitido!
Página 25: Acesso permitido!
Página 26: Acesso permitido!
Página 27: Acesso permitido!
Página 28: Acesso permitido!
Página 29: Acesso permitido!
Página 30: Acesso permitido!
Página 31: Acesso permitido!
Página 32: Acesso permitido!
Página 33: Acesso permitido!
Página 34: Acesso permitido!
Pág

### 

In [35]:
habilidade_raw.head()

,id,nome,geracao,pokemons
0,1,stench,generation-iii,"[gloom, grimer, muk, koffing, weezing, stunky,..."
1,2,drizzle,generation-iii,"[politoed, pelipper, kyogre]"
2,3,speed-boost,generation-iii,"[yanma, torchic, combusken, blaziken, ninjask,..."
3,4,battle-armor,generation-iii,"[cubone, marowak, kabuto, kabutops, anorith, a..."
4,5,sturdy,generation-iii,"[geodude, graveler, golem, magnemite, magneton..."


In [36]:
tipo_raw.head()

,id,nome,geracao,pokemons
0,1,normal,generation-i,"[pidgey, pidgeotto, pidgeot, rattata, raticate..."
1,2,fighting,generation-i,"[mankey, primeape, poliwrath, machop, machoke,..."
2,3,flying,generation-i,"[charizard, butterfree, pidgey, pidgeotto, pid..."
3,4,poison,generation-i,"[bulbasaur, ivysaur, venusaur, weedle, kakuna,..."
4,5,ground,generation-i,"[sandshrew, sandslash, nidoqueen, nidoking, di..."


In [37]:
habilidade_raw.head()

,id,nome,geracao,pokemons
0,1,stench,generation-iii,"[gloom, grimer, muk, koffing, weezing, stunky,..."
1,2,drizzle,generation-iii,"[politoed, pelipper, kyogre]"
2,3,speed-boost,generation-iii,"[yanma, torchic, combusken, blaziken, ninjask,..."
3,4,battle-armor,generation-iii,"[cubone, marowak, kabuto, kabutops, anorith, a..."
4,5,sturdy,generation-iii,"[geodude, graveler, golem, magnemite, magneton..."
